# PNN
- Product-based Neural Networks for User Response Prediction
- 参考：https://arxiv.org/abs/1611.00144

各个field间计算内、外积



In [ ]:
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding, Conv1D
from tensorflow.keras.regularizers import l2

In [ ]:
tag = fc.categorical_column_with_hash_bucket('tag', hash_bucket_size=20, dtype=tf.int64)
seq = fc.categorical_column_with_hash_bucket('seq', hash_bucket_size=10, dtype=tf.int64)
target = fc.categorical_column_with_hash_bucket('target', hash_bucket_size=10, dtype=tf.int64)
tag_col = fc.embedding_column(tag, dimension=8)
seq_col = fc.embedding_column(seq, dimension=8)
target_col = fc.embedding_column(target, dimension=8)
columns = [tag_col, seq_col, target_col]
features={
    "tag": tf.sparse.SparseTensor(
        indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
        values=[1100, 1101, 1102, 1101, 1103],
        dense_shape=[3, 2]),
    "seq": tf.sparse.SparseTensor(
        indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
        values=[1100, 1101, 1102, 1101, 1103],
        dense_shape=[3, 2]),
    "target": tf.sparse.SparseTensor(
        indices=[[0, 0],[1,0],[2,0]],
        values=[1102,1103,1100],
        dense_shape=[3, 1]),
}
tf.sparse.to_dense(features['seq'])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1100, 1101],
       [1102, 1101],
       [1103,    0]], dtype=int32)>

In [ ]:
feature_dict = {}
input_layer = tf.keras.layers.DenseFeatures(columns, name='features_input_layer')
net = input_layer(features, feature_dict)
tf.shape(net)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 3, 24], dtype=int32)>

In [ ]:
inputs = [tf.expand_dims(t, axis=1) for t in feature_dict.values()]
inputs

[<tf.Tensor: shape=(3, 1, 8), dtype=float32, numpy=
 array([[[-0.01416775, -0.17104809, -0.16381067, -0.15289606,
           0.20006178,  0.3378787 , -0.04711822,  0.2986418 ]],
 
        [[-0.1031516 , -0.46226966, -0.23939501, -0.08379643,
           0.12641546, -0.15812463, -0.2509806 ,  0.09572566]],
 
        [[-0.4147958 ,  0.36646494,  0.4097821 ,  0.13339798,
          -0.12564605,  0.27744687, -0.44212052, -0.24815147]]],
       dtype=float32)>, <tf.Tensor: shape=(3, 1, 8), dtype=float32, numpy=
 array([[[ 0.3819305 ,  0.2638193 ,  0.30913514, -0.28965178,
          -0.06341483,  0.03574949, -0.08744869, -0.2862989 ]],
 
        [[ 0.3289794 ,  0.10770745,  0.09811664, -0.01788428,
           0.05478749, -0.10102948, -0.06773475, -0.39072248]],
 
        [[ 0.52729   , -0.15069675, -0.05601599, -0.12594588,
           0.18221226, -0.04182186, -0.30024305, -0.45383793]]],
       dtype=float32)>, <tf.Tensor: shape=(3, 1, 8), dtype=float32, numpy=
 array([[[ 0.25513086, -0.078976

In [ ]:
def inner_pnn(inputs, use_reduce=False):
    f1_list, f2_list = [], []
    num_features = len(inputs)
    for f1 in range(num_features - 1):
        for f2 in range(f1 + 1, num_features):
            f1_list.append(f1)
            f2_list.append(f2)
    a = tf.concat([inputs[f] for f in f1_list], axis=1)
    b = tf.concat([inputs[f] for f in f2_list], axis=1)
    inner_product = a * b
    print(inner_product)
    if use_reduce:
        pnn_output = tf.reduce_sum(inner_product, axis=-1)
    else:
        pnn_output = tf.reshape(inner_product,[tf.shape(inner_product)[0], -1])
    return pnn_output

In [ ]:
inner_pnn(inputs)

tf.Tensor(
[[[-0.00541109 -0.04512579 -0.05063963  0.04428662 -0.01268688
    0.01207899  0.00412043 -0.08550082]
  [-0.00361463  0.01350883  0.05981699 -0.06571929  0.02219472
    0.18494418  0.01177508 -0.17678806]
  [ 0.09744225 -0.02083561 -0.11288358 -0.12450098 -0.0070352
    0.01956815  0.02185386  0.1694814 ]]

 [[-0.03393475 -0.04978989 -0.02348863  0.00149864  0.00692599
    0.01597525  0.01700011 -0.03740216]
  [ 0.03820502 -0.10777464 -0.07236038  0.02841793  0.08626803
   -0.00394944  0.07080386 -0.00754969]
  [-0.12184653  0.02511117  0.02965708  0.00606511  0.0373879
   -0.00252339  0.01910857  0.03081548]]

 [[-0.21871766 -0.05522508 -0.02295435 -0.01680093 -0.02289425
   -0.01160334  0.13274361  0.11262055]
  [-0.11583254 -0.00872965  0.08756232  0.05354417  0.02961279
   -0.17331223 -0.1691524  -0.04599581]
  [ 0.14724676  0.00358978 -0.01196951 -0.050553   -0.04294456
    0.02612478 -0.11487101 -0.08412057]]], shape=(3, 3, 8), dtype=float32)


<tf.Tensor: shape=(3, 24), dtype=float32, numpy=
array([[-0.00541109, -0.04512579, -0.05063963,  0.04428662, -0.01268688,
         0.01207899,  0.00412043, -0.08550082, -0.00361463,  0.01350883,
         0.05981699, -0.06571929,  0.02219472,  0.18494418,  0.01177508,
        -0.17678806,  0.09744225, -0.02083561, -0.11288358, -0.12450098,
        -0.0070352 ,  0.01956815,  0.02185386,  0.1694814 ],
       [-0.03393475, -0.04978989, -0.02348863,  0.00149864,  0.00692599,
         0.01597525,  0.01700011, -0.03740216,  0.03820502, -0.10777464,
        -0.07236038,  0.02841793,  0.08626803, -0.00394944,  0.07080386,
        -0.00754969, -0.12184653,  0.02511117,  0.02965708,  0.00606511,
         0.0373879 , -0.00252339,  0.01910857,  0.03081548],
       [-0.21871766, -0.05522508, -0.02295435, -0.01680093, -0.02289425,
        -0.01160334,  0.13274361,  0.11262055, -0.11583254, -0.00872965,
         0.08756232,  0.05354417,  0.02961279, -0.17331223, -0.1691524 ,
        -0.04599581,  0.14

In [ ]:
inner_pnn(inputs,True)

tf.Tensor(
[[[-0.00541109 -0.04512579 -0.05063963  0.04428662 -0.01268688
    0.01207899  0.00412043 -0.08550082]
  [-0.00361463  0.01350883  0.05981699 -0.06571929  0.02219472
    0.18494418  0.01177508 -0.17678806]
  [ 0.09744225 -0.02083561 -0.11288358 -0.12450098 -0.0070352
    0.01956815  0.02185386  0.1694814 ]]

 [[-0.03393475 -0.04978989 -0.02348863  0.00149864  0.00692599
    0.01597525  0.01700011 -0.03740216]
  [ 0.03820502 -0.10777464 -0.07236038  0.02841793  0.08626803
   -0.00394944  0.07080386 -0.00754969]
  [-0.12184653  0.02511117  0.02965708  0.00606511  0.0373879
   -0.00252339  0.01910857  0.03081548]]

 [[-0.21871766 -0.05522508 -0.02295435 -0.01680093 -0.02289425
   -0.01160334  0.13274361  0.11262055]
  [-0.11583254 -0.00872965  0.08756232  0.05354417  0.02961279
   -0.17331223 -0.1691524  -0.04599581]
  [ 0.14724676  0.00358978 -0.01196951 -0.050553   -0.04294456
    0.02612478 -0.11487101 -0.08412057]]], shape=(3, 3, 8), dtype=float32)


<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.1388782 ,  0.04611781,  0.0430903 ],
       [-0.10321546,  0.0320607 ,  0.0237754 ],
       [-0.10283145, -0.34230334, -0.12749732]], dtype=float32)>

## 结论
1. inner_pnn选择reduce，将点积后的结果聚合；
2. 不进行reduce，将得到emb；
3. 计算前[batch_size, 1, dims]，结果为[batch_size, N*(N-1)/2, (dims|1)]；
4. 每个样本得到N*(N-1)/2个交互结果